In [380]:
import cv2
import numpy as np
from PIL import Image
import dlib
import os

In [381]:
def load_img(path):
    img = cv2.imread(path)
    return img

In [382]:
def draw_predict(frame, left, top, right, bottom):
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)

In [383]:
def get_eyes_nose_dlib(shape):
    nose = shape[4][1]
    left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
    left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
    right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
    right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2
    return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

In [384]:
def get_eyes_nose(eyes, nose):
    left_eye_x = int(eyes[0][0] + eyes[0][2] / 2)
    left_eye_y = int(eyes[0][1] + eyes[0][3] / 2)
    right_eye_x = int(eyes[1][0] + eyes[1][2] / 2)
    right_eye_y = int(eyes[1][1] + eyes[1][3] / 2)
    nose_x = int(nose[0][0] + nose[0][2] / 2)
    nose_y = int(nose[0][1] + nose[0][3] / 2)

    return (nose_x, nose_y), (right_eye_x, right_eye_y), (left_eye_x, left_eye_y)

In [385]:
def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy

In [386]:
def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False

In [387]:
def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)


def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a

In [388]:
def show_img(img):
    while True:
        cv2.imshow('face_alignment_app', img)
        c = cv2.waitKey(1)
        if c == 27:
            break
    cv2.destroyAllWindows()

In [389]:
def shape_to_normal(shape):
    shape_normal = []
    for i in range(0, 5):
        shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
    return shape_normal


In [390]:
def rotate_opencv(img, nose_center, angle):
    M = cv2.getRotationMatrix2D(nose_center, angle, 1)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_CUBIC)
    return rotated

In [391]:
def rotation_detection_dlib(img):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat/shape_predictor_5_face_landmarks.dat')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    if len(rects) > 0:
        for rect in rects:
            x = rect.left()
            y = rect.top()
            w = rect.right()
            h = rect.bottom()
            shape = predictor(gray, rect)
            shape = shape_to_normal(shape)
            nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
            center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
            center_pred = (int((x + w) / 2), int((y + y) / 2))
            length_line1 = distance(center_of_forehead, nose)
            length_line2 = distance(center_pred, nose)
            length_line3 = distance(center_pred, center_of_forehead)
            cos_a = cosine_formula(length_line1, length_line2, length_line3)
            angle = np.arccos(cos_a)
            rotated_point = rotate_point(nose, center_of_forehead, angle)
            rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
            if is_between(nose, center_of_forehead, center_pred, rotated_point):
                angle = np.degrees(-angle)
            else:
                angle = np.degrees(angle)

            # img = img[y:h, x:w]
            img = rotate_opencv(img, nose, angle)

            gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces2 = detector(gray2)
            if len(faces2) > 0:
                for face in faces2:
                    x1, y1 = face.left(), face.top()
                    x2, y2 = face.right(), face.bottom()
                    img = img[y1:y2, x1:x2]
                    break
            else:
                return img, False
            break
        return img, True
    else:
        return img, False

In [392]:
def save_img(path, img):
    cv2.imwrite(path, img)

In [393]:
dir = "./smaller-data"
for filename in os.listdir(dir):
    filepath = (os.path.join(dir, filename))
    name = filename.split(".")
    name = name[0]

    videoFile = np.load(filepath)
    colorImages = videoFile['colorImages']
    boundingBox = videoFile['boundingBox']
    landmarks2D = videoFile['landmarks2D']

    height = colorImages.shape[0]
    width = colorImages.shape[1]
    size = (width, height)
    out = cv2.VideoWriter(os.path.join('./videos2',f'{name}.mp4'), cv2.VideoWriter_fourcc(*'h264'), 25, size, isColor=False)
    for i in range(colorImages.shape[-1]):
        viss = []
        img = np.ones((height, width, 3), np.uint8)
        img1 = colorImages[:, :, :, i]
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

        img1, t = rotation_detection_dlib(img1)
        img1 = cv2.resize(img1, (256, 256), interpolation = cv2.INTER_AREA)
        if t:
            out.write(img1)
            cv2.imshow('video', img1)
            c = cv2.waitKey(1)
            if c == 27:
                break
    out.release()


KeyboardInterrupt: 